**MDS7203 Modelos Generativos Profundos, Primavera 2023**

# Laboratorio 2: Modelo de lenguaje auto-regresivo

**Profesor**: Felipe Tobar, **Auxiliares**: Cristóbal Alcázar, Camilo Carvajal Reyes, **Ayudante**: Joaquín Barceló.

**Fecha de entrega**: viernes 29 de septiembre 2023

**Nombre: Sebastián Sanhueza**

**Instrucciones**: El presente notebook contiene enunciado e instrucciones para la realización del laboratorio. Usted deberá completar los códigos (en este archivo o en una copia del mismo) donde se le pida hacerlo. Usted deberá entregar el notebook con sus respuestas, cumpliendo lo siguiente:

- Los comentarios en código deben ser concisos pero claros. No se evaluarán sub-preguntas donde solo exista código sin comentarios pertinentes.
- El código debe ser ordenado y ejectuable. No se evaluarán notebooks o scripts que generen errores en su ejecución. Se aconseja resetear la kernel y corroborar la correcta execución de todas las celdas antes de ejecutar el entrenamiento de su modelo.
- Si bien se aconseja el uso de internet y otras herramientas para asistir su trabajo, asi como discusiones con el ED y estudiantes, el código que entregue debe ser de su autoría.

El objetivo del laboratorio será implementar, desde casi cero, un modelo de lenguaje estilo GPT, i.e., basado en el uso de un bloque "decoder" de la arquitectura Transformer (como se muestra en la imagen a continuación). Este tipo de modelos es un ejemplo de modelo auto-regresivo y que ha tenido gran relevancia en el último tiempo.

In [1]:
from IPython.display import Image
Image(url='https://i.stack.imgur.com/bWnx0.png')

Algunos links útiles:

* [Language Models are Unsupervised Multitask Learners](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf)
* [GPT2, original blog post](https://openai.com/research/better-language-models)

### Resumen de preguntas

- [ ] a) (0,5 ptos.) Definición de diccionarios para vocabulario

- [ ] b) (bonus) Utilización de embeddings previo a la normalización

- [ ] c) (bonus) Escalamiento por $1/\sqrt{d_k}$.

- [ ] d) (1.5 ptos.) Creación de clase `Head`.

- [ ] e) (0.75 pto.) Implementación de clase `FeedForward`.

- [ ] f) (0.5 ptos.) Relación entre hiper-parámetros n_head y head_size.

- [ ] g) (0.75 ptos.) Forward pass en `DecoderBlock`.

- [ ] h) (1 pto.) Implementación clase `GPTLM`.

- [ ] i) (1 pto.) Training loop.

- [ ] j) (bonus) Comparar modelo con Baseline.

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Características de la GPU, si es que está disponible.

In [3]:
!nvidia-smi

Sat Sep 30 05:13:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Corpus 📖

Escoja uno de los dos datasets:
- shakespeare.txt: concatenación de obras de shakespeare, ~ 1 millon de caracteres
- cabromagico.txt: concatenación de los libros de Harry Potter, ~ 6 millones de caracteres

Escoja en base a sus gustos y capacidades de cómputo.

In [4]:
# Si usan collab, descargar dataset desde repo GAMES descomentando una de las siguientes lineas:
# !wget https://raw.githubusercontent.com/GAMES-UChile/Curso-Modelos-Generativos-Profundos/main/labs/data/shakespeare.txt
!wget https://raw.githubusercontent.com/GAMES-UChile/Curso-Modelos-Generativos-Profundos/main/labs/data/cabromagico.txt

--2023-09-30 05:13:56--  https://raw.githubusercontent.com/GAMES-UChile/Curso-Modelos-Generativos-Profundos/main/labs/data/cabromagico.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6425820 (6.1M) [text/plain]
Saving to: ‘cabromagico.txt’

cabromagico.txt     100%[===================>]   6.13M  --.-KB/s    in 0.09s   

2023-09-30 05:13:57 (70.6 MB/s) - ‘cabromagico.txt’ saved [6425820/6425820]



In [5]:
# Selecciona el corpus a su gusto
#filename = 'shakespeare.txt'
filename = 'cabromagico.txt'

with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

print(f"Tamaño del corpus ({filename}): {len(text):,} caracteres")

Tamaño del corpus (cabromagico.txt): 6,340,988 caracteres


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"$%&'()*,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz|}~é–—‘’“”…　【】下为书件作你做全制区坛子式志您文新最本来格电的社立米糯自要论载
137


> a) (0.5 ptos.) Dada una lista de ordenada de caracteres, defina:
> - stoi: un diccionario caracter -> índice
> - itos: un diccionario índice -> caracter
> Con lo anterior, defina dos funciones encode y decode que tomen un string y una lista de índices respectivamente y devuelvan una lista de índices y un string según corresponda.

In [7]:
# --------------------------------------------

# Diccionario tipo caracter:índice
stoi = {char:index for index,char in enumerate(chars)}

# Diccionario tipo índice:caracter
itos = {index:char for index,char in enumerate(chars)}

# encoder: toma un string, devuelve una lista de índices
encode = lambda s: [stoi[element] for element in s]

# decoder: toma una lista de índices, devuelve un string
decode = lambda l: ''.join([itos[element] for element in l])

# --------------------------------------------

In [8]:
if filename == 'shakespeare.txt':
    assert encode('hola, que tal?') == [46, 53, 50, 39, 6, 1, 55, 59, 43, 1, 58, 39, 50, 12], 'Verifica que el output entregue una lista de enteros'
    assert decode(encode('hola, que tal?')) == 'hola, que tal?', 'Debe ser un string'
elif filename == 'cabromagico.txt':
    assert encode('hola, que tal?') == [73, 80, 77, 66, 14, 4, 82, 86, 70, 4, 85, 66, 77, 33], 'Verifica que el output entregue una lista de enteros'
    assert decode(encode('hola, que tal?')) == 'hola, que tal?', 'Debe ser un string'
else:
    print('Estas usando un filename distinto para construir el corpus!\nSi estas explorando otro corpus esta bien!\nRecuerda verificar que la funcionalidad esta correcta con shakespear o cabromagico.')

Nuestro modelo no entiende el lenguaje directamente, sino que los representa como números. Pasamos el corpus completo a su representación de enteros, usando el `stoi` (aka _string-to-index_).

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
data.shape

torch.Size([6340988])

In [10]:
N=100
print(f"Texto con los primeros {N} caracteres:\n----------------------------------------------\n")
print(text[:N])
print("\n----------------------------------------------\nSu representación como tensor de PyTorch...\n")
print(data[:N])

Texto con los primeros 100 caracteres:
----------------------------------------------

Harry Potter and the Sorcerer's Stone
CHAPTER ONE
THE BOY WHO LIVED
Mr. and Mrs. Dursley, of number 

----------------------------------------------
Su representación como tensor de PyTorch...

tensor([41, 66, 83, 83, 90,  4, 49, 80, 85, 85, 70, 83,  4, 66, 79, 69,  4, 85,
        73, 70,  4, 52, 80, 83, 68, 70, 83, 70, 83, 10, 84,  4, 52, 85, 80, 79,
        70,  1, 36, 41, 34, 49, 53, 38, 51,  4, 48, 47, 38,  1, 53, 41, 38,  4,
        35, 48, 58,  4, 56, 41, 48,  4, 45, 42, 55, 38, 37,  1, 46, 83, 16,  4,
        66, 79, 69,  4, 46, 83, 84, 16,  4, 37, 86, 83, 84, 77, 70, 90, 14,  4,
        80, 71,  4, 79, 86, 78, 67, 70, 83,  4])


## 2. Separar el dataset 🔨 y 🎓

In [11]:
n = int(0.9 * len(data))  # 90%
train_data = data[:n]
val_data = data[n:]
print(f"--> Tamaño del corpus de entrenamiento: {len(train_data):,} ({(train_data.shape[0] / data.shape[0]):.2f}) caracteres")
print(f"--> Tamaño del corpus de validación: {len(val_data):,} ({(val_data.shape[0] / data.shape[0]):.2f}) caracteres")

--> Tamaño del corpus de entrenamiento: 5,706,889 (0.90) caracteres
--> Tamaño del corpus de validación: 634,099 (0.10) caracteres


Sobre la mécanica de datos y etiquetas,

* Accedemos a los datos a partir de "fragmentos contextuales"; esto es un bloque de texto en representación númerica de tamaño `block_size`
* El modelo es semi-supervisado, es decir, búscamos entrenar un modelo de tal forma que dado ${x}_{i:j}$ _tokens_, vamos a predecir el siguiente _token_ $x_{j+1}$
* Las etiquetas emergen del mismo bloque contextual moviendo la ventana con un _offset_ de 1.

Por ejemplo, dado un bloque de tamaño 8,


In [12]:
block_size = 13
print(f"Una bloque contextual (X, Y) será:\n")
print(f"X: {[x.item() for x in data[:block_size]]}")
print(f"  --> decode(X): {decode([x.item() for x in data[:block_size]])}")
print('------------------------------------')
print(f"Y: {[x.item() for x in data[1:block_size+1]]}")
print(f"  --> decode(Y): {decode([y.item() for y in data[1:block_size+1]])}")

Una bloque contextual (X, Y) será:

X: [41, 66, 83, 83, 90, 4, 49, 80, 85, 85, 70, 83, 4]
  --> decode(X): Harry Potter 
------------------------------------
Y: [66, 83, 83, 90, 4, 49, 80, 85, 85, 70, 83, 4, 66]
  --> decode(Y): arry Potter a


Sin embago, dentro de cada bloque contextual ocupamos la información de manera autoregresiva, generando múltiple observaciones a partir de este...

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Cuando el input es {context} el target es: {target}")

Cuando el input es tensor([41]) el target es: 66
Cuando el input es tensor([41, 66]) el target es: 83
Cuando el input es tensor([41, 66, 83]) el target es: 83
Cuando el input es tensor([41, 66, 83, 83]) el target es: 90
Cuando el input es tensor([41, 66, 83, 83, 90]) el target es: 4
Cuando el input es tensor([41, 66, 83, 83, 90,  4]) el target es: 49
Cuando el input es tensor([41, 66, 83, 83, 90,  4, 49]) el target es: 80
Cuando el input es tensor([41, 66, 83, 83, 90,  4, 49, 80]) el target es: 85
Cuando el input es tensor([41, 66, 83, 83, 90,  4, 49, 80, 85]) el target es: 85
Cuando el input es tensor([41, 66, 83, 83, 90,  4, 49, 80, 85, 85]) el target es: 70
Cuando el input es tensor([41, 66, 83, 83, 90,  4, 49, 80, 85, 85, 70]) el target es: 83
Cuando el input es tensor([41, 66, 83, 83, 90,  4, 49, 80, 85, 85, 70, 83]) el target es: 4
Cuando el input es tensor([41, 66, 83, 83, 90,  4, 49, 80, 85, 85, 70, 83,  4]) el target es: 66


Por lo tanto, cada bloque contextual, genera un número de observaciones igual a su tamaño.

En términos de _batches_, podemos procesar en paralelo, múltiples bloques contextuales. Lo importante es que cada bloque contextual es independiente, y no hay computo que ocurra a nivel transversal, sino paralelo entre estos. No se mezclan las secuencias autoregresivas de cada contexto.

In [14]:
# colocar seed como su RUT
torch.manual_seed(206684402)
batch_size = 4
block_size = 8  # largo de ventana máximo para considerar en la precisión
# Estos parámetros se re-definirán para el entrenamiento final

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"Cuando el input es {context.tolist()} el target: {target}")

inputs:
torch.Size([4, 8])
tensor([[  6,   4,   1,   6,  47,  80,  14,   6],
        [  4,  85,  80,  80,  33,   6,   1, 103],
        [ 71,   4,  71,  80,  86,  83,   4,  73],
        [ 73,  66,  85,  33,   6,   1,   6,  34]])
targets:
torch.Size([4, 8])
tensor([[  4,   1,   6,  47,  80,  14,   6,   4],
        [ 85,  80,  80,  33,   6,   1, 103, 103],
        [  4,  71,  80,  86,  83,   4,  73,  80],
        [ 66,  85,  33,   6,   1,   6,  34,   4]])
----
Cuando el input es [6] el target: 4
Cuando el input es [6, 4] el target: 1
Cuando el input es [6, 4, 1] el target: 6
Cuando el input es [6, 4, 1, 6] el target: 47
Cuando el input es [6, 4, 1, 6, 47] el target: 80
Cuando el input es [6, 4, 1, 6, 47, 80] el target: 14
Cuando el input es [6, 4, 1, 6, 47, 80, 14] el target: 6
Cuando el input es [6, 4, 1, 6, 47, 80, 14, 6] el target: 4
Cuando el input es [4] el target: 85
Cuando el input es [4, 85] el target: 80
Cuando el input es [4, 85, 80] el target: 80
Cuando el input es [4, 85, 80, 

Lo que recibirá la red como _input_ será:

In [15]:
xb

tensor([[  6,   4,   1,   6,  47,  80,  14,   6],
        [  4,  85,  80,  80,  33,   6,   1, 103],
        [ 71,   4,  71,  80,  86,  83,   4,  73],
        [ 73,  66,  85,  33,   6,   1,   6,  34]])

## 3. Baseline

Creamos un modelo base clásico, para luego compararlo con nuestro Transformer.

In [16]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # cada token lee sucesivamente los logits para el token siguiente de una lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx y targets son ambos tensores de tamaño (B,T) con elementos enteros
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx es un arreglo (B, T) de indices del contexto actual
        for _ in range(max_new_tokens):
            # obtener predicciones
            logits, loss = self(idx)
            # concentrarse en el último paso
            logits = logits[:, -1, :]  # se convierte en (B, C)
            # aplicamos softmax para obtener probabilidades
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # samplear de la distribución
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # agregar el índice de la muestra a la secuencia actual
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 137])
tensor(5.6164, grad_fn=<NllLossBackward0>)
	全【'下d‘立”区Mt为=iag:,来Iq的?Uh1你的xB'D<>】CPjK文1ZCU你Ye全为\糯'i(^l*
Jh论社2您做x'}&?[^B书Z!95:[T2`您nt社d4格e下”~4HVr3?


## 4. Self-attention

En la sección anterior, vimos que cada _token_ toma una representación vectorial llamada _embeddings_.

Nuestro corpus contiene $65$ caracteres únicos con un _embedding_ asociado a cada _token_. La idea de representar el lenguaje en términos de tokens, y estos a su vez en vectores, es que podemos aprender estas representaciones vectoriales a partir de los datos. Sin embargo, la representación es única, y muchas veces un mismo _token_ puede tener distintos significados según su contexto. Por ejemplo:
1. "Te banco a morir!"
2. "El banco está abierto hasta las dos."

En ambas oraciones anteriores, el token `banco` tiene un significado distinto. Se espera entonces que la representación de ese token sea distinta en ambos casos y eso lo logramos con la influencia de los tokens presentes en el mismo contexto.

La idea principal de _self-attention_ es utilizar la secuencia de _embeddings_ dentro de un contexto para computar un promedio ponderado a partir de estos. Dado una secuencia de _embeddings_ de _tokens_ $x_1, \dots, x_n$, el mecanismo de _self-attention_ (o auto-atención) produce una nueva secuencia de _embeddings_ $x'_1, \dots, x'_n$, donde cada $x'_i$ es una combinación lineal de todos los $x_j$:

$$
x'_i = \sum_{j=1}^{n} \alpha_{ij} x_{j}
$$

Los coeficientes $\alpha_{ij}$ se llaman ponderadores de atención y están normalizados tal que $\sum_{j}\alpha_{ji}=1$.

En términos sencillos, construiremos un mecanismo de comunicación entre distintos tokens dentro del bloque de contexto, que se representará por una colección de ponderadores en una matriz. Esta colección de ponderadores la llamaremos matriz de atención (o self-attention) y nos permitirá vía la operación de multiplicación de matrices, agregar distintos valores dentro de un bloque contextual en una sola cantidad. Spoiler, estos pesos serán data-dependientes.

Comencemos emulando la operación con pesos fijos, usaremos la parte triangular inferior de una matriz identidad de 3x3, la cual normalizaremos a nivel de fila.

In [17]:
# Ejemplo de juguete que ilustra como la multiplicación matricial puede ser usada para una adición con pesos
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print('a=',a,'\n')
print('b=',b,'\n')
print('c=',c)

a= tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) 

b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) 

c= tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


Notemos que $c$ tiene en cada fila los resultados de los valores acumulados de $b$ según los ponderadores de $a$.

El tensor $a$ se interpreta como una matriz de token-a-token y representa la interación/influencia del token en la posición $i$ con el token de la posición $j$. Dado que nuestro modelo es autoregresivo, los tokens del presente solo pueden ser influenciados por tokens pasados, o ellos mismos. Por eso las posiciones de $a$ que cumplen esta restricción $i \leq j$, son elementos que conforman la matriz triangular inferior de $a$. El resto de las posiciones no tiene influencia sobre los tokens pasados (i.e. 0).


Vamos a crear un _batch_ con datos síntetico de tamaño `B`, donde cada bloque contextual será de largo $T$, y cada _token_ que compone el contexto se representa por $C$ dimensiones (i.e. tamaño del _embedding_).

In [18]:
torch.manual_seed(1337)

B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [19]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [20]:
# Versión usando softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [21]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

Los ponderadores anteriores son uniformes, ahora introducimos los conceptos de _queries_ y _keys_ para ver como computar los ponderadores a partir de los datos.

* Un _query_ $q(\cdot)$ corresponde a una proyección lineal de la representación de _embeddings_ de un token particular. Por ejemplo, se proyecto $\mathbb{R}^{C}\rightarrow \mathbb{R}^{H}$.
* Los _keys_ es la matriz $K\in\mathbb{R}^{T\times H}$ que contiene proyecciones lineales de todos los _embeddings_ de tokens dentro del contexto, incluído el token que es el query. La proyección lineal de los _keys_ es de igual tamaño (i.e. $H$) que la proyección del _query_.
* Los ponderadores para cada _query_ se obtiene a partir de qué tan próximo se relaciona un token respecto al resto de los token dentro de un contexto. Por ejemplo, $q(x_i) \times K$

In [22]:
Image(url="https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/context-vector.png")

> b) (Bonus): Explique porqué no se utilizan directamente los _embeddings_ para computar la matriz de atención previo a la normalización, i.e. `X @ X.transpose(-2,-1)`, en vez de usar las proyecciones $QK^\top$. $X$ es un tensor con dimensiones batch size (B), ventana de contexto (T), dimensiones de embedding (C).

Esto es debido a que las proyecciones permiten que el modelo aprenda representaciones más significativas de los tokens de entrada y permite ajustarse a cualquier tarea, ya que se adaptan a través del entrenamiento. Esto es muy importante en tareas de procesamiento de lenguaje natural, ya que el aprender representaciones de buena manera puede tener un impacto significativo en el rendimiento del modelo, además el permitir que el modelo pueda ajustarse a cualquier tarea permite que con, por ejemplo, tareas de traducción de texto, el modelo pueda aprender a enfocarse más en ciertas partes de la secuencia de entrada utilizando la información contextual.

Por otro lado, si se utiliza directamente los _embeddings_ se perderán las ventajas ofrecidas por el uso de las proyecciones, en particular no permitiría que el modelo aprendiera representaciones más específicas de los datos de entrada para la atención.

> c) (Bonus): Explique los argumentos detras de escalar por $1/\sqrt{d_k}$ referidos en el paper _[Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)_ (Vaswani 2017).

La aplicación del escalado por $1/\sqrt{d_k}$ asegura que los vectores de peso mantengan una longitud euclidiana de magnitud similar. Esto ayuda a evitar que los pesos de atención se vuelvan excesivamente pequeños o grandes, lo cual puede provocar problemas de inestabilidad numérica o afectar a la capacidad del modelo para converger durante el entrenamiento.

In [22]:
# colocar seed como su RUT
torch.manual_seed(206684402)

B,T,C = 4,8,32  # batch, time, channels
x = torch.randn(B,T,C)

Ejemplo de aplicación de módulo de atención

In [23]:
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
wei =  q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

Observaciones:
- Atención es un **mecanismo de comunicación**. Puede ser entendido como nodos en un grafo dirigido conectándose unos con otros y agregando información con una suma ponderada de todos los nodos que apuntan a ellos, con pesos dependientes de los datos.
- No hay una noción de espacio. Atención simplemente actua sobre el conjunto de vectores. Es por este motivo que se necesitan encoders posicionales.
- Cada punto dentro de un batch es, desde luego, procesado de manera independiente y nunca intractua con los otros.
- En un bloque de atención "encoder" basta comentar la linea que hace masking con `tril`, que hace que los tokens se comuniquen todos con todos. El bloque anterior se llama "decoder" porque aplica un masking triangular y se encuentre frecuentemente en configuraciones autoregresivas.
- "auto-atención" (_self-attention_) sólo signfica que tanto _keys_ como _values_ son producidas desde la misma fuente que las _queries_. En "atención-cruzada" (_cross-attention_), las _queries_ vienen de $x$, pero _keys_ y _values_ vienen de otra fuente externa (como puede ser un modulo encoder).
- Atención "escalada" divide `wei` por $\frac{1}{\sqrt{head\_size}}$. Esto hace que cuando los input $Q$ y $K$ tengan varianza unitaria, `wei` también tendrá varianza unitaria y evitará la saturación de la Softmax.

> d) (1.5 ptos.) Cree una clase `Head` que implemente un módulo de auto-atención.

In [24]:
n_embd = 64  # dimensionalidad del input
dropout = 0.0

class Head(nn.Module):
    """ Una cabeza de auto-atención """

    def __init__(self, head_size):
        super().__init__()
        # ------------------------

        self.head_size = head_size # Variable que define el tamaño de la cabeza de atención

        # Inicialización las matrices de key, query y value respectivamente, mediante el uso de capas lineales
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        # ------------------------
        # HINT: cuando aplique tril, ocupe self.tril se define automaticamente
        # al instanciar
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input size (batch, time-step, channels)
        # output size (batch, time-step, channels)
        B,T,C = x.shape
        # -----------------------------------------

        # Aplicación de las matrices de pesos sobre la secuencia de entrada,
        # se generan las secuencias de key, query y values respectivamente
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        v = self.value(x)  # (B,T,C)

        # Computar los score de atención ("affinities")

        # Se calculan los productos punto entre querys y keys
        wei = (q @ k.transpose(-2, -1))/(self.head_size**0.5)  # (B, T, C) @ (B, C, T) -> (B, T, T)

        # Se aplica la máscara triangular inferior
        wei = wei.masked_fill_(self.tril == 0, float('-inf'))   # (B, T, T)

        # Se aplica la función Softmax para obtener las probabilidades o pesos de atención
        wei = F.softmax(wei, dim=-1)  # (B, T, T)

        # Se aplica el método de Dropout
        wei = self.dropout(wei)

        # Adición (con pesos) de las atenciones
        # Se realiza el producto entre los pesos de atención y la secuencia de value
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)

        # --------------------------------------

        return out

La arquitectura decoder del paper Transformer implementa varias versiones de _self-attention_ en paralelo, cada una es una "cábeza de atención", y estas concatenan sus resultados en un modulo conocido como `MultiHeadAttention`.

In [25]:
class MultiHeadAttention(nn.Module):
    """ Múltiples cabezas de auto-atención en paralelo """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

> e) (0.75 pto.) Implemente una clase `FeedForward` como se describe en el artículo ["Attention is all you need, Vaswani et al."](https://arxiv.org/pdf/1706.03762.pdf).

In [26]:
class FeedFoward(nn.Module):
    """
        Implementar FeedForward descrita en sección:
         "3.3 Position-wise Feed-Fordward Networks", paper
         "Attention is All You Need"
        https://arxiv.org/pdf/1706.03762.pdf

        in: n_embd
        out: n_embd
    """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            # -------------------

            # La red de Feed-Forward consiste en dos transformaciones lineales
            # con una función de activación ReLU en medio de las dos.

            # La primera transforamción lineal aumenta el tamaño del input en 4
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            # La segunda transformación lineal vuelve a la dimensionalidad original
            nn.Linear(4 * n_embd, n_embd)

            # ------------------
        )

    def forward(self, x):
        return self.net(x)


In [28]:
Image(url="http://jalammar.github.io/images/gpt2/gpt2-transformer-block-vectors-2.png")

> f) (0.5 ptos.) Explique la relación entre los hiperparámetros `n_head` y `head_size` de la clase `MultiHeadAttention`. Piense en su rol dentro del bloque decoder (i.e. atención + feedforward).

Respuesta: Al analizar la clase `MultiHeadAttention` se puede observar que la entrada se divide en `n_head`cabezas de atención independientes, donde cada una de ellas tiene su propio conjunto de proyecciones(Q, K y V) que se aplican a la entrada original. La dimensionalidad de las proyecciones Q, K y V en cada cabeza de atención es controlada por el hiperparámetro `head_size`. Por lo tanto, cada cabeza de atención tiene una dimensión `head_size` para sus proyecciones.

Además, dentro de cada cabeza de atención, se realiza la atención entre las proyecciones Q, K y V y se obtienen las salidas ponderadas. Luego, las salidas de todas las cabezas de atención se concatenan para formar una representación combinada que tiene una dimensión total de n_head * head_size.

Por otra parte, en la capa de feedforward, la entrada es el resultado que proviene de la atención multi-cabeza, al cual la capa de feedforward aplica transformaciones lineales y transformaciones no lineales como funciones de activación(ReLU) para obtener las representaciones finales antes de la salida.

Ahora procederemos a armar la clase que implemente un bloque de Decoder. Nótese que en realidad estamos codificando texto con este bloque, pero esto corresponde a la parte "Decoder" del Transformer original, por ende guardamos esa nomenclatura. La motivación del uso del decoder es modelar el texto de maner auto-regresiva, que es ideal para la generación de texto.

In [29]:
Image(url='https://i.stack.imgur.com/bWnx0.png')

> g) (0.75 ptos.) Complete el paso _forward_ de la clase `DecoderBlock`. Recuerde en particula incorporar las conexiones residuales (_skip connections_).

In [27]:
class DecoderBlock(nn.Module):
    """  BloqueTransformer: COMUNICACIÓN seguida de CÓMPUTO """

    def __init__(self, n_embd, n_head):
        # n_embd: dimensión de embeddings, n_head: número de cabezas de atención
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Hint: aplique las capas de normalización siempre antes de otra capa (Pre-LN varaint)
        # Ver: https://magazine.sebastianraschka.com/p/why-the-original-transformer-figure
        # ---------------------

        # Aplicación de primera capa de normalización antes de la atención
        ln1_output = self.ln1(x)

        # Aplicación de capa de atención multi-cabeza
        sa_output = self.sa(ln1_output)

        # Conexión residual con la entrada original
        x = x + sa_output

        # Aplicación de segunda capa de normalización antes de la atención
        ln2_output = self.ln2(x)

        # Aplicación de capa de FeedForward
        ffwd_output = self.ffwd(ln2_output)

        # Conexión residual con la salida de la capa de FeedForward
        x = x + ffwd_output

        # ----------------------

        return x

## 5. Modelo GPT: Juntando todo

> h) (1 pto.) Complete el código de la clase `GPTLanguageModel`procesando adecuadamente el input del modelo. Complete además el método `generate` para samplear elementos que completen auto-regresivamente una secuencia.

In [28]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # cada token lee directamente los logits para el token siguiente de una lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[DecoderBlock(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx y targets son ambos tensores (B,T) de enteros
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)

        # -----------------------------

        # Combinación de los embeddings de token y posición
        x = tok_emb + pos_emb # (B,T,C)

        # Procesamiento de x a través de los bloques
        x = self.blocks(x)

        # Aplicamos la última capa de normalización
        x = self.ln_f(x)

        # Calculo de logits
        logits = self.lm_head(x) # (B,T,vocab_size)

        # -------------------------------

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx es un arreglo (B, T) de índices en el contexto actual
        for _ in range(max_new_tokens):
            # restringir idx a los últimos block_size tokens
            idx_cond = idx[:, -block_size:]
            # obtener las predicciones
            logits, loss = self(idx_cond)
            # enfocarse sólo en el último paso
            logits = logits[:, -1, :]  # se convierte en size (B, C)
            # ------------------------------------------------------------

            # Aplicación de función Softmax para obtener las probabilidades
            probs = F.softmax(logits, dim=-1)  # tensor de dimensionalidad (B, C)

            # Sampleo de la dsitribución multinomial usando las probabilidades
            idx_next = torch.multinomial(probs, num_samples=1)  # tensor de índices de dim (B, 1)

            # Actualización del índice a la secuencia actual
            idx = torch.cat([idx, idx_next], dim=1)  # tensor resultante de dimensionalidad (B, T+1)

            # ------------------------------------------------------------

        return idx


In [29]:
# Definimos hiperparámetros (global variables, esto se puede hacer mucho mejor)

batch_size = 64 # cuantos secuencias de fragmentos del corpus procesaremos de manera independiente (aka B)?
block_size = 256 # cuál será el tamaño del bloque de contexto a considerar para predecir (aka T)?
max_iters = 8000
eval_interval = 500
learning_rate = 3e-4  # modelo de tamaño pequeño probar esta lr por defecto
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384  # se debe considerar en conjunto con n_head, según análisis en (f)
n_head = 6    # se debe considerar en conjunto con n_embd, según análisis en (f)
n_layer = 6
dropout = 0.2


model = GPTLanguageModel()
model.to(device)

# printear el número de parámetros del modelo
print('Número de parámetros del modelo:', sum(p.numel() for p in model.parameters())/1e6, 'millones')

# definir el optimizador de PyTorch
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Número de parámetros del modelo: 10.844297 millones


Definiremos una función para generar batches de secuencias a partir de nuestro corpus de entrenamiento o validación. Además de una función para obtener estimaciones de la función de costo.

In [30]:
# definimos un "DataLoader"
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


# definimos una función para obtener estimados de nuestras
# pérdidas tanto en los conjuntos de train como en val
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Primer sanity check, obtener batches con dimensiones correctas:

In [31]:
# Verifiquemos que las dimensiones sean correctas
xb, yb = get_batch('train')
xb.shape, yb.shape

(torch.Size([64, 256]), torch.Size([64, 256]))

Segundo sanity check, verificar que la data fluya correctamente por el modelo,

In [32]:
# Verifiquemos que el forwardpass del modelo no tenga problemas
model(xb)[0].shape

torch.Size([64, 256, 137])

> i) (1 pto.) Complete el bucle de entrenamiento usando comandos de `pytorch.optimize` conocidos. Ejecute el entrenamiento (se recomienda dejarlo corriendo e ir hacer algo más...). Corrobore que se modelo es capaz de generar texto.

In [33]:
# colocar RUT como semilla
torch.manual_seed(206684402)

# comenzamos el training loop...
for iter in range(max_iters):

    # de vez en cuando evaluar la loss en los conjuntos de train y evaluación
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        context = torch.zeros((1, 256), dtype=torch.long, device=device)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        print("Testing text generation:")
        print(decode(model.generate(context, max_new_tokens=100)[0].tolist()))

    # samplear un batch de datos
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)

    # ----------------------------------------

    # Activación del optimizador
    # Limpieza de los gradientes acumulados
    optimizer.zero_grad()

    # Calcula los gradientes aplicando el paso backward con el resultado de la función loss
    loss.backward()

    # Actualización de los parámetros del modelo
    optimizer.step()

    # ----------------------------------------

# Se especifica la ruta y el nombre del archivo donde se quiere guardar el modelo

path = '/content/modelo_gpt.pth'

# Se guarda el modelo completo y entrenado, tanto la arquitectura como los parámetros

torch.save(model, path)

step 0: train loss 4.9346, val loss 4.9391
Testing text generation:
																																																																																																																																																																																																																																																																论MtUqsq,子制7最^:9的\}米F'hE全米Q件最论]”1^社Gx><论为,!5T`v1Q米hj-自UC\'做Y>|坛格\	自’k论R】W;&3Nd.】自d’‘3式本j<【做Y志UCCo-子l
step 500: train loss 1.5831, val loss 1.6544
Testing text generation:
																																																																																																																																																																																																																																																																UVeard.ILriykewitgle twenstaille tekor in upneesched as agre Of Pernofess
Ruckus at Harry,tilone"ll 
step 1000: train loss 1.2686, val loss 1.3673
Testing text generation:
																																																																													

## 6. Generando secuencias de texto con el modelo

In [34]:
# Se especifica la ruta y el nombre del archivo con el modelo que se quiere cargar

path = '/content/modelo_gpt.pth' # Se debe tener el archivo en el directorio indicado, si es distinta la dirección se debe editar

# Se carga el modelo completo

model = torch.load(path)

# Se establece el modo evaluación

model.eval()

GPTLanguageModel(
  (token_embedding_table): Embedding(137, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): DecoderBlock(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((384,), 

In [36]:
# Generar usando el modelo
context = torch.zeros((1, 256), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

# Para escribir en un archivo
# open('output.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

																																																																																																																																																																																																																																																																
802 He saideteseadis 

  

 “Hothappenet to you. . Wouldn’ve been on 
your oldest?” 

 “I don’t care that boy smart!” 

 Both Ron and Hermione had to look so dis-solved. 

 “My Lord,” Harry heard something on the table; he was sure it was hammering 

Xenophilius Flamel, blinking his, into a bowl of your rest instead, would never a 
sound of telephone in Mr. 


Cleanen this meant arrange that everyone are. There was long them beside Scrimgeour. He learned them all around on his 
matter? Where h


In [47]:
# Prueba con prompt escrito
prompt = torch.tensor(encode("Felipe Tobar, the new Defence Against the Dark Arts teacher, presented his new spell to his students. It was called MCMC Algorithm, which was known to be an attack so powerful that no one could survive it. Even the wizard Andrew Ng couldn't fight its power"), dtype=torch.long, device=device).unsqueeze(0)
print(decode(model.generate(prompt, max_new_tokens=5000)[0].tolist()))

Felipe Tobar, the new Defence Against the Dark Arts teacher, presented his new spell to his students. It was called MCMC Algorithm, which was known to be an attack so powerful that no one could survive it. Even the wizard Andrew Ng couldn't fight its powerful.
　　"WE won't go a letter, it's good," he was hoisted him. "What was that?"
　　"What?" said Harry in a very interest. "Good thinking dark. . .
　　But then - I'll add to them -"
　　"I don't like that news," said Dumbledore. "Flee was better talk to you to King's Cross St at all the other Minister's job, that Harry is not our telling Dark as much more as I know that. You are sure ere there."
　　"About your way," said Dumbledore. "You see? Harry Potter blinks it weren't my office. Think on our Knowing Crouch is affects that. You always have Filch sayed it in Dumbledore's a fat of old, but not even if you have already the one you will tell me you that might be at the hint of a panicg in the back of you. Dumbledore will tell you have decide

In [48]:
# Generamos un archivo de texto a partir del prompt
open('output.txt', 'w').write(decode(model.generate(prompt, max_new_tokens=10000)[0].tolist()))

10256

> j) (bonus) Define un training loop para el baseline (modelo bi-grama). Entrénelo usando un npumero similar de épocas y compare las losses y generación de texto con su modelo anterior.